In [ ]:
!git clone https://github.com/d2l-ai/d2l-zh.git

Cloning into 'd2l-zh'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 17279 (delta 26), reused 24 (delta 5), pack-reused 17185
Receiving objects: 100% (17279/17279), 185.22 MiB | 36.93 MiB/s, done.
Resolving deltas: 100% (12059/12059), done.


In [ ]:
import torch
import numpy as np

%matplotlib inline
from IPython import display
import matplotlib.pyplot as plt
import random

import torch.utils.data as Data

import torch.nn as nn
from torch.nn import init

import torch.optim as optim
from collections import OrderedDict


import torchvision
import torchvision.transforms as transforms
import time
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_pytorch
# import d2lzh_pytorch as d2l

import torchtext.vocab as Vocab
import os
from tqdm import tqdm
import tarfile
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATA_ROOT = "/S1/CSCL/tangss/Datasets"


In [ ]:
import zipfile
import os
data_root = '/content/d2l-zh/data'
fname = os.path.join(data_root, 'ptb.zip')
with zipfile.ZipFile(fname, 'r') as f:
  f.extractall(data_root)

In [ ]:
import collections
assert 'ptb.train.txt' in os.listdir(data_root + "/ptb")
import time, math
with open('/content/d2l-zh/data/ptb/ptb.train.txt', 'r') as f:
  lines = f.readlines() #每個elem為每行
  raw_dataset = [st.split() for st in lines] #每行中的每個elem為每個字
counter = collections.Counter([tk for st in raw_dataset for tk in st]) #return dict{words:count_nums}
counter = dict(filter(lambda x:x[1]>=5,counter.items())) 

idx_to_token = [tk for tk, _ in counter.items()] #idx => idx_to_token[idx] return token
token_to_idx = {tk:idx for idx, tk in enumerate(idx_to_token)} #token => token_to_idx[tk] return idx
dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx] for st in raw_dataset]
def discard(idx): #當任意機率<丟棄機率，就為True(準備丟棄)
  return random.uniform(0, 1) < 1 - math.sqrt(1e-4/counter[idx_to_token[idx]]*sum([len(st) for st in dataset]))

subsampled_dataset = [[tk for tk in st if not discard(tk)] for st in dataset]